9. Pessoas que tem o plano Premium fazem churn com menos frequência


In [ ]:
from google.colab import auth
import pandas as pd

# get credentials
auth.authenticate_user()
query = '''
SELECT user_id, churner, plan FROM `e-tensor-411113.neobankclean.userdevice_clean`
'''

df = pd.read_gbq(query, project_id="e-tensor-411113")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17512 entries, 0 to 17511
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  17512 non-null  object
 1   churner  17512 non-null  Int64 
 2   plan     17512 non-null  object
dtypes: Int64(1), object(2)
memory usage: 427.7+ KB


In [ ]:
pivot = pd.pivot_table(df,
                       values=['churner','plan'],
                       index=['churner'], columns=['plan'], aggfunc='count',
                       dropna=True)

In [ ]:
pivot

user_id                                                       
plan      METAL METAL_FREE PREMIUM PREMIUM_FREE PREMIUM_OFFER STANDARD
churner                                                               
0         390.0       21.0   755.0          4.0          17.0  13720.0
1           NaN        NaN    11.0          NaN           NaN   2594.0

In [ ]:
pivot=pivot.fillna(0)

In [ ]:
pivot

user_id                                                       
plan      METAL METAL_FREE PREMIUM PREMIUM_FREE PREMIUM_OFFER STANDARD
churner                                                               
0         390.0       21.0   755.0          4.0          17.0  13720.0
1           0.0        0.0    11.0          0.0           0.0   2594.0

In [ ]:
df['churner'].value_counts()

churner
0    14907
1     2605
Name: count, dtype: Int64

In [ ]:
# 11/2605 ~= 0,4%
# dentre os churners, 0,4% são premium users

# 755/14907 ~= 5%
# dentre os not churners, 5% são premium users

HO: Não há relação entre Premium users e churners

H1: Há relação entre Premium users e churners

In [ ]:
import scipy.stats as stats

# Teste Chi quadrado
chi_res = stats.chi2_contingency(pivot)

print(f'p-value: {chi_res[1]}')

p-value: 6.602064177321436e-41




*   0 <= p-value <= 1

Se p-value < valor de significância: H0 é rejeitada. Fixando o valor de significância em 0.05 (valor comumente usado na literatura), temos:
*   p-value < 0.05: H0 rejeitada ⇒ H1 aceita
*   p-value=> 0.05: H0 aceita ⇒ H1 rejeitada